In [ ]:
import h5py
from wildfire.data_utils import *
from wildfire.data_types import *
from wildfire.dataset_utils import *
import matplotlib.pyplot as plt
from einops import reduce
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cartopy.crs as ccrs
import cartopy.feature as cfeature

is_test = False
viirs = h5_get_path("viirs", is_test)
print(viirs)
viirs = h5py.File(viirs, "r")
modis = h5_get_path("modis", is_test)
modis = h5py.File(modis, "r")
dates = list(viirs["num_fire_pixels_by_day"].keys())

fires_path = os.path.join(config.root_path, "fires", "fires.json")
fires = json_load(fires_path)

# Convert coordinates into bbox format [min_lon, min_lat, max_lon, max_lat]
bbox = [146.37825193592488, -38.054848543980675, 153.93684568592488, -28.56311327381567]
gxs = int((bbox[2] - bbox[0]) / Resolution.KM + 1)
gys = int((bbox[3] - bbox[1]) / Resolution.KM + 1)

def one_fire(fire, ds, patch_size_org, global_img):
    patch_size = config.patch_size_km
    # Divide by 3 to get km resolution
    px0, py0, px1, py1 = [x // 3 for x in fire["pixel_bounds"]]
    xs = (px1 - px0) + 1
    ys = (py1 - py0) + 1
    gx0, gy0, gx1, gy1 = fire["grid_bounds"]
    bounds = fire["bounds"]
    bb_offset = [x - bbox[i % 2] for i, x in enumerate(bounds)]
    gpx0, gpy0, gpx1, gpy1 = [round(x / Resolution.KM) for x in bb_offset]
    gpy0 = gys - 1 - gpy1
    if gpx0 < 0 or gpy0 < 0 or gpx0 + xs >= gxs or gpy0 + ys >= gys:
        return
    start_date = fire["start_date"]
    end_date = fire["end_date"]
    t0 = dates.index(start_date)
    t1 = dates.index(end_date)
    default_val = np.full((patch_size_org, patch_size_org), 0, dtype=np.int16)
    def get_day(t, y, x, day_type):
        xy_path = H5Grid.get_cell_path(x, y)
        day = h5_get_nested(ds, ["cells", xy_path, dates[t], day_type, "fire_mask"])
        if day is None:
            return default_val
        return day[...][0]
    num_fires = 0
    img = np.full((ys, xs), 255, dtype=np.int16)
    for x in range(gx0, gx1 + 1):
        for y in range(gy0, gy1 + 1):
            for t in range(t0, t1 + 1):
                day = get_day(t, y, x, "day")
                night = get_day(t, y, x, "night")
                num_fires += np.sum(day > 6) + np.sum(night > 6)
                new = np.maximum(day, night)
                if patch_size != new.shape[-1]:
                    new = reduce(new, "(h 3) (w 3) -> h w", "max", h=patch_size, w=patch_size)
                xo = (x - gx0) * patch_size - px0
                yo = (gy1 - y) * patch_size - py0
                fill = np.full((patch_size, patch_size), 255, dtype=np.int16)
                fill[new > 6] = (t - t0)
                x0 = max(0, xo)
                y0 = max(0, yo)
                x1 = min(xs - 1, xo + patch_size)
                y1 = min(ys - 1, yo + patch_size)
                if x1 - x0 <= 0 or y1 - y0 <= 0:
                    continue
                old = img[y0:y1, x0:x1]
                fill = fill[y0 - yo:y1 - yo, x0 - xo:x1 - xo]
                res = np.minimum(old, fill)
                img[y0:y1, x0:x1] = res
    img[img == 255] = -1

    global_img[gpy0:gpy0 + img.shape[0], gpx0:gpx0 + img.shape[1]] = img


dir = os.path.join(config.root_path, "figures", "fires_global")
os.makedirs(dir, exist_ok=True)

# for i in range(i0, i0 + 5):
viirs_global = np.full((gys, gxs), -1, dtype=np.int16)
modis_global = np.full((gys, gxs), -1, dtype=np.int16)
for fire in fires[:100]:
    one_fire(fire, viirs, config.patch_size, viirs_global)
    one_fire(fire, modis, config.patch_size, modis_global)

projection = ccrs.PlateCarree() 
fig, ax = plt.subplots(1, 2, subplot_kw={"projection": projection}, figsize=(6, 3.6))

background = np.ones((gys, gxs, 3)) * 0.85

bbox2 = [bbox[0], bbox[2], bbox[1], bbox[3]]
vmax = max(viirs_global.max(), modis_global.max()) - 10

def geo_axis(ax, data):
    ax.set_extent(bbox2, crs=projection)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=":")
    ax.add_feature(cfeature.LAND, facecolor="lightgray")
    # Mark Sydney and Canberra
    sydney = [151.2093, -33.8688]
    canberra = [149.1300, -35.2809]
    ax.imshow(background, extent=bbox2, transform=projection)
    im = ax.imshow(
        data.astype(np.float32), cmap="YlOrRd_r",
        vmin=0, vmax=vmax, extent=bbox2, transform=projection
    )
    ax.plot(sydney[0], sydney[1], 'g*', markersize=8, transform=projection)
    ax.text(sydney[0] + 0.2, sydney[1]-0.13, 'Sydney', transform=projection)
    ax.plot(canberra[0], canberra[1], 'g*', markersize=8, transform=projection) 
    ax.text(canberra[0] - 1.2, canberra[1]+0.27, 'Canberra', transform=projection)
    return im

viirs_global = np.ma.masked_where(viirs_global == -1, viirs_global)
modis_global = np.ma.masked_where(modis_global == -1, modis_global)

im = geo_axis(ax[0], viirs_global)
ax[0].set_title("VNP14", fontsize=12)


im = geo_axis(ax[1], modis_global)
ax[1].set_title("MOD14", fontsize=12)

fig.tight_layout(rect=[0, 0, 0.88, 1])

# Create a dedicated colorbar axis.
cax = fig.add_axes([0.88, 0.08, 0.02, 0.8])
fig.colorbar(im, cax=cax, label="Days since fire started")

out_path = f"{dir}/fire_global.pdf"
fig.savefig(out_path, dpi=150)
plt.show()
print(out_path)

